# First Test for EVT

In [1]:
import yfinance as yf
import numpy as np
from arch import arch_model
from itertools import product

# Import functions
import backtesting as BT
import plots as IP
import evt as EVT

### Data

In [2]:
# Download data
sp500_data = yf.download("^GSPC", start="2000-01-01", end="2024-01-01")
sp500_data["Log Returns"] = np.log(sp500_data["Close"] / sp500_data["Close"].shift(1))
returns = sp500_data["Log Returns"].dropna()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


### EVT

In [3]:
# Set parameters
confidence_level = 0.99 # <----- Can choose 0.95 etc
threshold_percentile = 99

In [4]:
# Apply EVT model
evt_output = EVT.evt_var(
    returns=returns,
    confidence_level=confidence_level,
    threshold_percentile=threshold_percentile
    )

evt_output.head()


,Returns,VaR,VaR Violation
Date,,,
2000-01-04,-0.039099,0.03515,True
2000-01-05,0.001920,0.03515,False
2000-01-06,0.000955,0.03515,False
2000-01-07,0.026730,0.03515,False
2000-01-10,0.011128,0.03515,False


In [5]:
# Plot interactive VaR
fig_var = IP.plot_backtest(evt_output, subset=("2020-01-01", "2025-01-01"), interactive=True)


In [6]:
# Trying ES
evt_output = EVT.evt_es(
    result_data=evt_output,
    threshold_percentile=threshold_percentile
    )

evt_output.head()   

,Returns,VaR,VaR Violation,ES
Date,,,,
2000-01-04,-0.039099,0.03515,True,0.051492
2000-01-05,0.001920,0.03515,False,0.051492
2000-01-06,0.000955,0.03515,False,0.051492
2000-01-07,0.026730,0.03515,False,0.051492
2000-01-10,0.011128,0.03515,False,0.051492


In [7]:
# Plot interactive VaR and ES
fig_var = IP.plot_backtest(evt_output, subset=("2020-01-01", "2025-01-01"), interactive=True)


### Backtesting (Extensive)

In [8]:
total_violations, violation_rate = BT.count_violations(evt_output)

print(f"Total VaR Violations: {total_violations}")
print(f"Violation Rate: {violation_rate:.2f}%")


Total VaR Violations: 59
Violation Rate: 0.98%


In [9]:
kupiec_results = BT.kupiec_test(
    total_violations=total_violations,
    total_days=len(evt_output),
    confidence_level=confidence_level
)

print("\nKupiec Test Results:")
for key, value in kupiec_results.items():
    print(f"{key}: {value}")


Kupiec Test Results:
LR_uc: 0.031185080569116508
p_value: 0.8598281086079077
reject_null: False


In [10]:
christoffersen_results = BT.christoffersen_test(evt_output)

print("\nChristoffersen Test Results:")
for key, value in christoffersen_results.items():
    print(f"{key}: {value}")



Christoffersen Test Results:
LR_c: 9.180378493283001
p_value: 0.0024462343073486936
reject_null: True


In [11]:
joint_results = BT.joint_lr_test(
    LR_uc=kupiec_results["LR_uc"],
    LR_c=christoffersen_results["LR_c"]
)

print("\nJoint Test Results:")
for key, value in joint_results.items():
    print(f"{key}: {value}")


Joint Test Results:
LR_total: 9.211563573852118
p_value: 0.009993885860517637
reject_null: True
